# RAG 시스템을 더 똑똑하게: Adaptive RAG
이 노트북에서는 질문의 성격에 따라 가장 적합한 검색 전략을 유연하게 선택하는 '적응형 검색 시스템'을 구현합니다. 이를 통해 RAG 시스템은 훨씬 더 정확하고 맥락에 잘 맞는 답변을 제공할 수 있게 됩니다.

질문의 종류에 따라, 필요한 정보 접근 방식은 달라질 수밖에 없습니다. 그래서 Adaptive RAG에서는 다음과 같은 과정을 따릅니다:
1. 질문이 어떤 유형인지 분류합니다 (사실 기반, 분석적, 의견형, 맥락 중심 등)
2. 그에 맞는 검색 전략을 선택합니다.
3. 상황에 특화된 검색 기법을 실행합니다.
4. 질문에 꼭 맞는 방식으로 답변을 구성합니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv
import re

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('OPENAI_API_KEY')

# OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [ ]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.  

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## Simple Vector Store 구축
NumPy를 사용하여 간단한 Vecotr store 구축

In [ ]:
import numpy as np

class SimpleVectorStore:
    """
    NumPy를 사용하여 간단한 Vecotr store 구축
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목 추가

        Args:
        text (str): 원본 텍스트.
        embedding (List[float]): 임베딩 벡터.
        metadata (dict, optional): 추가 메타데이터.
        """
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def similarity_search(self, query_embedding, k=5, filter_func=None):
        """
        쿼리 임베딩과 가장 유사한 항목 찾기.

        Args:
        query_embedding (List[float]): 쿼리 임베딩 벡터.
        k (int): 반환할 결과의 수.
        filter_func (callable, optional): 결과를 필터링하는 함수.

        Returns:
        List[Dict]: 텍스트와 메타데이터가 포함된 상위 k개 유사 항목.
        """
        if not self.vectors:
            return []  # 벡터가 저장되어 있지 않으면 빈 리스트 반환
        
        # 쿼리 임베딩을 numpy 배열로 변환
        query_vector = np.array(query_embedding)
        
        # 코사인 유사도를 사용하여 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            # 필터가 제공된 경우 적용
            if filter_func and not filter_func(self.metadata[i]):
                continue
                
            # 코사인 유사도 계산
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 추가
        
        # 유사도(내림차순)로 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 텍스트 추가
                "metadata": self.metadata[idx],  # 메타데이터 추가
                "similarity": score  # 유사도 점수 추가
            })
        
        return results  # 상위 k개 결과 반환

## 임베딩 생성

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

## Document Processing Pipeline

In [ ]:
def process_document(file_path, chunk_size=1000, chunk_overlap=200):
    """
    문서를 처리하여 Adaptive RAG에 적합한 형태로 변환합니다.

    Args:
    file_path (str): 파일 경로
    chunk_size (int): 각 청크의 크기
    chunk_overlap (int): 청크 간 중복 범위

    Returns:
    Tuple[List[str], SimpleVectorStore]: 문서 청크와 벡터 저장소.
    """
    # PDF 파일에서 텍스트 추출
    # print("Extracting text from PDF...")
    # extracted_text = extract_text_from_pdf(pdf_path)

    # 텍스트 파일 로드
    extracted_text = load_text_file(file_path)
    
    # 추출된 텍스트를 청크로 분할
    print("Chunking text...")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"Created {len(chunks)} text chunks")
    
    # 텍스트 청크의 임베딩 생성
    print("Creating embeddings for chunks...")
    chunk_embeddings = create_embeddings(embedding_model, chunks, device=device, batch_size=4)
    
    # 벡터 저장소 생성
    store = SimpleVectorStore()
    
    # 각 청크와 임베딩을 VectorStore에 저장
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={"index": i, "source": file_path}
        )
    
    print(f"Added {len(chunks)} chunks to the vector store")
    
    # 청크와 벡터 저장소 반환
    return chunks, store

## Query Classification

In [ ]:
def classify_query(query, model_name="gpt-4.1-mini"):
    """
    쿼리를 다음 네 가지 유형 중 하나로 분류합니다.
    - Factual, Analytical, Opinion, or Contextual.
    
    Args:
        query (str): 사용자 쿼리
        model_name (str): 사용할 모델
        
    Returns:
        str: Query category
    """
    # 시스템 프롬프트
    system_prompt = """You are an expert at classifying questions. 
        Classify the given query into exactly one of these categories:
        - Factual: Queries seeking specific, verifiable information.
        - Analytical: Queries requiring comprehensive analysis or explanation.
        - Opinion: Queries about subjective matters or seeking diverse viewpoints.
        - Contextual: Queries that depend on user-specific context.

        Return ONLY the category name, without any explanation or additional text.
    """

    # 사용자 프롬프트
    user_prompt = f"Classify this query: {query}"
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 분류 결과 추출
    category = response.choices[0].message.content.strip()
    
    # 유효한 분류 결과 목록 정의
    valid_categories = ["Factual", "Analytical", "Opinion", "Contextual"]
    
    # 생성된 분류 결과가 유효한지 확인
    for valid in valid_categories:
        if valid in category:
            return valid
    
    # 분류 실패 시 "Factual"로 기본값 설정
    return "Factual"

## Implementing Specialized Retrieval Strategies
### 1. 사실 기반 전략 – 정확성에 집중

In [ ]:
def factual_retrieval_strategy(query, vector_store, k=4):
    """
    정확성에 중점을 둔 사실 기반 쿼리에 대한 검색 전략.
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서의 수
        
    Returns:
        List[Dict]: 검색된 문서
    """
    print(f"Executing Factual retrieval strategy for: '{query}'")
    
    # 시스템 프롬프트
    system_prompt = """You are an expert at enhancing search queries.
        Your task is to reformulate the given factual query to make it more precise and 
        specific for information retrieval. Focus on key entities and their relationships.

        Provide ONLY the enhanced query without any explanation.
    """

    user_prompt = f"Enhance this factual query: {query}"
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 변환된(향상된) 쿼리 추출 및 출력
    enhanced_query = response.choices[0].message.content.strip()
    print(f"Enhanced query: {enhanced_query}")
    
    # 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [enhanced_query], device=device, batch_size=1)[0]
    
    # 유사도 검색을 통해 문서 검색
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)
    
    ranked_results = []
    
    # LLM을 사용하여 문서 관련성 점수 계산 및 순위 매기기
    for doc in initial_results:
        relevance_score = score_document_relevance(enhanced_query, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "relevance_score": relevance_score
        })
    
    # 관련성 점수에 따라 결과 정렬
    ranked_results.sort(key=lambda x: x["relevance_score"], reverse=True)
    
    # 상위 k개 결과 반환
    return ranked_results[:k]

### 2. Analytical Strategy - 폭넓은 정보 제공

In [ ]:
def analytical_retrieval_strategy(query, vector_store, k=4):
    """
    종합적인 정보 획득을 위한 분석 쿼리에 대한 검색 전략
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서의 수
        
    Returns:
        List[Dict]: 검색된 문서
    """
    print(f"Executing Analytical retrieval strategy for: '{query}'")
    
    # 시스템 프롬프트 
    system_prompt = """You are an expert at breaking down complex questions.
    Generate sub-questions that explore different aspects of the main analytical query.
    These sub-questions should cover the breadth of the topic and help retrieve 
    comprehensive information.

    Return a list of exactly 3 sub-questions, one per line.
    """

    # 사용자 프롬프트
    user_prompt = f"Generate sub-questions for this analytical query: {query}"
    
    # 서브 쿼리 생성
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )
    
    # 생성된 서브 쿼리 추출
    sub_queries = response.choices[0].message.content.strip().split('\n')
    sub_queries = [q.strip() for q in sub_queries if q.strip()]
    print(f"Generated sub-queries: {sub_queries}")
    
    # 서브 쿼리로 문서 검색
    all_results = []
    for sub_query in sub_queries:
        sub_query_embedding = create_embeddings(embedding_model, [sub_query], device=device, batch_size=1)[0]
        # 유사 문서 검색
        results = vector_store.similarity_search(sub_query_embedding, k=2)
        all_results.extend(results)
    
    # 서브 쿼리 결과에서 다양성 보장
    # 중복 제거 (동일한 텍스트 내용)
    unique_texts = set()
    diverse_results = []
    
    for result in all_results:
        if result["text"] not in unique_texts:
            unique_texts.add(result["text"])
            diverse_results.append(result)
    
    # k개의 결과를 얻기 위해 초기 결과에서 더 많은 결과 추가
    if len(diverse_results) < k:
        # 메인 쿼리에 대한 직접 검색
        main_query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
        main_results = vector_store.similarity_search(main_query_embedding, k=k)
        
        for result in main_results:
            if result["text"] not in unique_texts and len(diverse_results) < k:
                unique_texts.add(result["text"])
                diverse_results.append(result)
    
    # 상위 k개 다양한 결과 반환
    return diverse_results[:k]

### 3. Opinion Strategy - 댜앙한 관점

In [ ]:
def opinion_retrieval_strategy(query, vector_store, k=4):
    """
    다양한 관점을 반영하기 위해, 의견형 질문에는 균형 잡힌 시각을 담은 자료들을 폭넓게 수집하는 검색 전략을 사용합니다.
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서의 수
        
    Returns:
        List[Dict]: 검색된 문서
    """
    print(f"Executing Opinion retrieval strategy for: '{query}'")
    
    # 시스템 프롬프트
    system_prompt = """You are an expert at identifying different perspectives on a topic.
        For the given query about opinions or viewpoints, identify different perspectives 
        that people might have on this topic.

        Return a list of exactly 3 different viewpoint angles, one per line.
    """

    # 사용자 프롬프트 생성
    user_prompt = f"Identify different perspectives on: {query}"
    
    # 다양한 관점 생성
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3
    )
    
    # 관점 추출
    viewpoints = response.choices[0].message.content.strip().split('\n')
    viewpoints = [v.strip() for v in viewpoints if v.strip()]
    print(f"Identified viewpoints: {viewpoints}")
    
    # 각 viewpoint을 나타내는 문서 검색
    all_results = []
    for viewpoint in viewpoints:
        # 메인 쿼리와 viewpoint을 결합
        combined_query = f"{query} {viewpoint}"
        # 결합된 쿼리의 임베딩 생성
        viewpoint_embedding = create_embeddings(embedding_model, [combined_query], device=device, batch_size=1)[0]
        # 결합된 쿼리에 대한 유사도 검색
        results = vector_store.similarity_search(viewpoint_embedding, k=2)
        
        # 결과에 관점을 표시
        for result in results:
            result["viewpoint"] = viewpoint
        
        # 결과 추가
        all_results.extend(results)
    
    # 다양한 관점 선택
    selected_results = []
    for viewpoint in viewpoints:
        viewpoint_docs = [r for r in all_results if r.get("viewpoint") == viewpoint]
        if viewpoint_docs:
            selected_results.append(viewpoint_docs[0])
    
    # 아직 채워지지 않은 자리를 유사도가 가장 높은 문서들로 채움.
    remaining_slots = k - len(selected_results)
    if remaining_slots > 0:
        # 남은 문서들을 유사도 기준으로 정렬
        remaining_docs = [r for r in all_results if r not in selected_results]
        remaining_docs.sort(key=lambda x: x["similarity"], reverse=True)
        selected_results.extend(remaining_docs[:remaining_slots])
    
    # 상위 k개 결과 반환
    return selected_results[:k]

### 4. Contextual Strategy - User Context 통합

In [ ]:
def contextual_retrieval_strategy(query, vector_store, k=4, user_context=None):
    """
    사용자 맥락을 통합한 맥락 기반 쿼리에 대한 검색 전략
    
    Args:
        query (str): 사용자 쿼리
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 반환할 문서의 수
        user_context (str): 추가 사용자 맥락
        
    Returns:
        List[Dict]: 검색된 문서
    """
    print(f"Executing Contextual retrieval strategy for: '{query}'")
    
    # 사용자 맥락이 제공되지 않으면 쿼리에서 유추
    if not user_context:
        system_prompt = """You are an expert at understanding implied context in questions.
For the given query, infer what contextual information might be relevant or implied 
but not explicitly stated. Focus on what background would help answering this query.

Return a brief description of the implied context."""

        user_prompt = f"Infer the implied context in this query: {query}"
        
        # Generate the inferred context using the LLM
        response = client_openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )
        
        # inferred context 추출 및 출력
        user_context = response.choices[0].message.content.strip()
        print(f"Inferred context: {user_context}")
    
    # 사용자 context를 포함하도록 쿼리 재구성
    system_prompt = """You are an expert at reformulating questions with context.
    Given a query and some contextual information, create a more specific query that 
    incorporates the context to get more relevant information.

    Return ONLY the reformulated query without explanation."""

    user_prompt = f"""
    Query: {query}
    Context: {user_context}

    Reformulate the query to incorporate this context:"""
    
    # LLM을 사용하여 맥락화된 쿼리 생성
    response = client_openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 맥락화된 쿼리 추출 및 출력
    contextualized_query = response.choices[0].message.content.strip()
    print(f"Contextualized query: {contextualized_query}")
    
    # 유사 문서 검색
    query_embedding = create_embeddings(embedding_model, [contextualized_query], device=device, batch_size=1)[0]
    initial_results = vector_store.similarity_search(query_embedding, k=k*2)
    
    # 관련성과 사용자 맥락을 고려하여 문서 순위 매기기
    ranked_results = []
    
    for doc in initial_results:
        # 맥락을 고려하여 문서 관련성 점수 계산 
        context_relevance = score_document_context_relevance(query, user_context, doc["text"])
        ranked_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "similarity": doc["similarity"],
            "context_relevance": context_relevance
        })
    
    # 맥락 관련성에 따라 결과 정렬
    ranked_results.sort(key=lambda x: x["context_relevance"], reverse=True)
    return ranked_results[:k]

## Helper Functions for Document Scoring

In [ ]:
def score_document_relevance(query, document, model_name="gpt-4.1-mini"):
    """
    LLM을 사용하여 문서 관련성 점수 계산.
    
    Args:
        query (str): User query
        document (str): Document text
        model (str): LLM model
        
    Returns:
        float: 0-10 범위의 관련성 점수
    """
    # 시스템 프롬프트
    system_prompt = """You are an expert at evaluating document relevance.
        Rate the relevance of a document to a query on a scale from 0 to 10, where:
        0 = Completely irrelevant
        10 = Perfectly addresses the query

        Return ONLY a numerical score between 0 and 10, nothing else.
    """

    # 문서가 너무 길면 truncate
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document
    
    # 쿼리와 문서 미리보기가 포함된 사용자 프롬프트
    user_prompt = f"""
        Query: {query}

        Document: {doc_preview}

        Relevance score (0-10):
    """
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 응답에서 점수 추출
    score_text = response.choices[0].message.content.strip()
    
    # 점수 추출
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 0-10 범위로 제한
    else:
        # 추출 실패 시 기본 점수
        return 5.0

In [ ]:
def score_document_context_relevance(query, context, document, model_name="gpt-4.1-mini"):
    """
    쿼리와 맥락을 고려하여 문서 관련성 점수 계산
    
    Args:
        query (str): User query
        context (str): User context
        document (str): Document text
        model (str): LLM model
        
    Returns:
        float: 0-10 범위의 관련성 점수
    """
    # 시스템 프롬프트
    system_prompt = """You are an expert at evaluating document relevance considering context.
        Rate the document on a scale from 0 to 10 based on how well it addresses the query
        when considering the provided context, where:
        0 = Completely irrelevant
        10 = Perfectly addresses the query in the given context

        Return ONLY a numerical score between 0 and 10, nothing else.
    """

    # 문서가 너무 길면 truncate
    doc_preview = document[:1500] + "..." if len(document) > 1500 else document
    
    # 쿼리, 맥락, 문서 미리보기가 포함된 사용자 프롬프트
    user_prompt = f"""
    Query: {query}
    Context: {context}

    Document: {doc_preview}

    Relevance score considering context (0-10):
    """
    
    # 응답 생성 
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 응답에서 점수 추출 
    score_text = response.choices[0].message.content.strip()
    
    # 점수 추출
    match = re.search(r'(\d+(\.\d+)?)', score_text)
    if match:
        score = float(match.group(1))
        return min(10, max(0, score))  # 0-10 범위로 제한
    else:
        # 추출 실패 시 기본 점수
        return 5.0

## The Core Adaptive Retriever

In [ ]:
def adaptive_retrieval(query, vector_store, k=4, user_context=None):
    """
    적절한 전략을 선택하여 적응형 검색 수행
    
    Args:
        query (str): 사용자 쿼리    
        vector_store (SimpleVectorStore): 벡터 저장소
        k (int): 검색할 문서 수
        user_context (str): 상황에 따라, 문맥형 질문에 사용자 정보를 선택적으로 반영할 수 있음
    Returns:
        List[Dict]: Retrieved documents
    """
    # 쿼리 유형 분류
    query_type = classify_query(query)
    print(f"Query classified as: {query_type}")
    
    # 쿼리 유형에 따라 적절한 검색 전략 선택 및 실행
    if query_type == "Factual":
        # 정확한 정보를 위한 사실 기반 검색 전략
        results = factual_retrieval_strategy(query, vector_store, k)
    elif query_type == "Analytical":
        # 종합적인 정보 획득을 위한 분석 쿼리에 대한 검색 전략
        results = analytical_retrieval_strategy(query, vector_store, k)
    elif query_type == "Opinion":
        # 다양한 관점을 위한 의견 기반 검색 전략
        results = opinion_retrieval_strategy(query, vector_store, k)
    elif query_type == "Contextual":
        # 사용자 맥락을 포함하는 맥락 기반 검색 전략
        results = contextual_retrieval_strategy(query, vector_store, k, user_context)
    else:
        # 분류 실패 시 기본 사실 기반 검색 전략
        results = factual_retrieval_strategy(query, vector_store, k)
    
    return results  # 검색된 문서 반환

## Response Generation

In [ ]:
def generate_response(query, results, query_type, model_name="gpt-4.1-nano"):
    """
    질문 내용, 검색된 문서들, 그리고 질문의 유형을 바탕으로 가장 적절한 답변을 생성합니다.
    
    Args:
        query (str): User query
        results (List[Dict]): Retrieved documents
        query_type (str): Type of query
        model_name (str): LLM model
        
    Returns:
        str: Generated response
    """
    # 검색된 문서의 텍스트를 구분자로 결합하여 맥락 준비
    context = "\n\n---\n\n".join([r["text"] for r in results])
    
    # 쿼리 유형에 따라 시스템 프롬프트 생성
    if query_type == "Factual":
        system_prompt = """You are a helpful assistant providing factual information.
    Answer the question based on the provided context. Focus on accuracy and precision.
    If the context doesn't contain the information needed, acknowledge the limitations."""
        
    elif query_type == "Analytical":
        system_prompt = """You are a helpful assistant providing analytical insights.
    Based on the provided context, offer a comprehensive analysis of the topic.
    Cover different aspects and perspectives in your explanation.
    If the context has gaps, acknowledge them while providing the best analysis possible."""
        
    elif query_type == "Opinion":
        system_prompt = """You are a helpful assistant discussing topics with multiple viewpoints.
    Based on the provided context, present different perspectives on the topic.
    Ensure fair representation of diverse opinions without showing bias.
    Acknowledge where the context presents limited viewpoints."""
        
    elif query_type == "Contextual":
        system_prompt = """You are a helpful assistant providing contextually relevant information.
    Answer the question considering both the query and its context.
    Make connections between the query context and the information in the provided documents.
    If the context doesn't fully address the specific situation, acknowledge the limitations."""
        
    else:
        system_prompt = """You are a helpful assistant. Answer the question based on the provided context. If you cannot answer from the context, acknowledge the limitations."""
    
    # 컨텍스트와 쿼리를 결합하여 사용자 프롬프트 생성
    user_prompt = f"""
    Context:
    {context}

    Question: {query}

    Please provide a helpful response based on the context.
    """
    
    # 응답 생성
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    
    # 생성된 응답 내용 반환
    return response.choices[0].message.content

## Complete RAG Pipeline with Adaptive Retrieval

In [ ]:
def rag_with_adaptive_retrieval(file_path, query, k=4, user_context=None):
    """
    Adaptive RAG 파이프라인
    
    Args:
        file_path (str): 파일 경로
        query (str): 사용자 쿼리
        k (int): 반환할 문서의 수
        user_context (str): 선택적 사용자 맥락
        
    Returns:
        Dict: Results including query, retrieved documents, query type, and response
    """
    print("\n=== RAG WITH ADAPTIVE RETRIEVAL ===")
    print(f"Query: {query}")
    
    # 문서를 처리하여 텍스트 추출, 청크로 나누고 임베딩 생성
    chunks, vector_store = process_document(file_path)
    
    # 쿼리 유형 분류
    query_type = classify_query(query)
    print(f"Query classified as: {query_type}")
    
    # 쿼리 유형에 따라 적절한 검색 전략 선택 및 실행
    retrieved_docs = adaptive_retrieval(query, vector_store, k, user_context)
    
    # 쿼리, 검색된 문서, 쿼리 유형에 따라 응답 생성
    response = generate_response(query, retrieved_docs, query_type)
    
    # 질문에 대한 전체 결과 저장
    result = {
        "query": query,
        "query_type": query_type,
        "retrieved_documents": retrieved_docs,
        "response": response
    }
    
    print("\n=== RESPONSE ===")
    print(response)
    
    return result

## Evaluation Framework

In [ ]:
def evaluate_adaptive_vs_standard(pdf_path, test_queries, reference_answers=None):
    """
    테스트 쿼리 세트에 대해 adaptive RAG와 Standard RAG를 비교합니다.

    Args:
        pdf_path (str): 파일 경로
        test_queries (List[str]): 평가할 쿼리 목록
        reference_answers (List[str], optional): 평가 메트릭을 위한 참조 답변
        
    Returns:
        Dict: 개별 쿼리 결과와 전체 비교 결과가 포함된 평가 결과
    """
    print("=== EVALUATING ADAPTIVE VS. STANDARD RETRIEVAL ===")
    
    # 문서를 처리하여 텍스트 추출, 청크로 나누고 벡터 저장소 생성
    chunks, vector_store = process_document(pdf_path)
    
    # 비교 결과를 저장할 컬렉션 초기화
    results = []
    
    # 각 테스트 쿼리에 대해 Standard 검색과 adaptive 검색 모두 처리
    for i, query in enumerate(test_queries):
        print(f"\n\nQuery {i+1}: {query}")
        
        # --- Standard retrieval approach ---
        print("\n--- Standard Retrieval ---")
        # 쿼리의 임베딩 생성
        query_embedding = create_embeddings(embedding_model, [query], device=device, batch_size=1)[0]
        # 단순 벡터 유사도를 사용하여 문서 검색
        standard_docs = vector_store.similarity_search(query_embedding, k=4)
        # 일반적인 접근 방식을 사용하여 응답 생성
        standard_response = generate_response(query, standard_docs, "General")
        
        # --- Adaptive retrieval approach ---
        print("\n--- Adaptive Retrieval ---")
        # 쿼리 유형 분류 (Factual, Analytical, Opinion, Contextual)
        query_type = classify_query(query)
        # 쿼리 유형에 따라 적절한 검색 전략 선택 및 실행
        adaptive_docs = adaptive_retrieval(query, vector_store, k=4)
        # 쿼리 유형에 맞는 응답 생성
        adaptive_response = generate_response(query, adaptive_docs, query_type)
        
        # 쿼리에 대한 완전한 결과 저장
        result = {
            "query": query,
            "query_type": query_type,
            "standard_retrieval": {
                "documents": standard_docs,
                "response": standard_response
            },
            "adaptive_retrieval": {
                "documents": adaptive_docs,
                "response": adaptive_response
            }
        }
        
        # 참조 답변이 있는 경우 추가
        if reference_answers and i < len(reference_answers):
            result["reference_answer"] = reference_answers[i]
            
        results.append(result)
        
        # 빠른 비교를 위한 두 응답의 미리보기 표시
        print("\n--- Responses ---")
        print(f"Standard: {standard_response[:200]}...")
        print(f"Adaptive: {adaptive_response[:200]}...")
    
    # 기준 답안이 있을 경우, 비교 분석을 위한 지표를 산출합니다.
    if reference_answers:
        comparison = compare_responses(results)
        print("\n=== EVALUATION RESULTS ===")
        print(comparison)
    
    # 완전한 평가 결과 반환
    return {
        "results": results,
        "comparison": comparison if reference_answers else "No reference answers provided for evaluation"
    }

In [ ]:
def compare_responses(results):
    """
    표준 검색과 적응형 검색 응답을 참조 답변과 비교.
    
    Args:
        results (List[Dict]): Results containing both types of responses
        
    Returns:
        str: Comparison analysis
    """
    # 시스템 프롬프트
    comparison_prompt = """You are an expert evaluator of information retrieval systems.
    Compare the standard retrieval and adaptive retrieval responses for each query.
    Consider factors like accuracy, relevance, comprehensiveness, and alignment with the reference answer.
    Provide a detailed analysis of the strengths and weaknesses of each approach."""
    
    # 비교 텍스트 초기화
    comparison_text = "# Evaluation of Standard vs. Adaptive Retrieval\n\n"
    
    # 각 결과를 비교
    for i, result in enumerate(results):
        # 참조 답변이 없는 경우 건너뛰기
        if "reference_answer" not in result:
            continue
            
        # 쿼리 세부 정보를 비교 텍스트에 추가   
        comparison_text += f"## Query {i+1}: {result['query']}\n"
        comparison_text += f"*Query Type: {result['query_type']}*\n\n"
        comparison_text += f"**Reference Answer:**\n{result['reference_answer']}\n\n"
        
        # 표준 검색 응답을 비교 텍스트에 추가
        comparison_text += f"**Standard Retrieval Response:**\n{result['standard_retrieval']['response']}\n\n"
        
        # 적응형 검색 응답을 비교 텍스트에 추가
        comparison_text += f"**Adaptive Retrieval Response:**\n{result['adaptive_retrieval']['response']}\n\n"
        
        # 응답을 비교하기 위한 사용자 프롬프트 생성
        user_prompt = f"""
        Reference Answer: {result['reference_answer']}
        
        Standard Retrieval Response: {result['standard_retrieval']['response']}
        
        Adaptive Retrieval Response: {result['adaptive_retrieval']['response']}
        
        Provide a detailed comparison of the two responses.
        """
        
        # 비교 분석 생성
        response = client_openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": comparison_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2
        )
        
        # AI가 생성한 비교 분석 내용을 기존의 비교 텍스트에 추가합니다.
        comparison_text += f"**Comparison Analysis:**\n{response.choices[0].message.content}\n\n"
    
    return comparison_text 

## Evaluating the Adaptive Retrieval System (Customized Queries)

The final step to use the adaptive RAG evaluation system is to call the evaluate_adaptive_vs_standard() function with your PDF document and test queries:

In [ ]:
import pandas as pd
# 평가 데이터 로드
df = pd.read_csv('./data_creation/rag_val_new_post.csv')

# 테스트 쿼리 정의
test_queries = [
    "연금소득의 분리과세 기준금액은 얼마로 인상되었습니까?",# Factual query - seeking definition/specific information
    # "조정대상조세 금액의 합계액이 음수일 때, 추가세액비율이 최저한세율을 초과하는 문제를 어떻게 해소할 수 있습니까?" # analsis query - requiring comprehensive analysis
    # "가상자산 과세 유예 조치는 납세자 보호 측면에서 정당하다고 볼 수 있을까요?", # Opinion query - seeking diverse perspectives
    # "여자 코일은 선행발명의 코일과 동일한가요?", # Contextual query - benefits from context-awareness
]

# 참조 답변 정의
reference_answers = [
    "연금소득의 분리과세 기준금액은 1천200만원에서 1천500만원으로 인상되었습니다.",
    # "조정대상조세 금액의 합계액이 음수일 때, 추가세액비율이 최저한세율을 초과하는 문제를 해소하기 위해 해당 사업연도 실효세율을 영(零)으로 보고, 이 금액은 해당 사업연도 실효세율 계산에 산입됩니다.",
    # "가상자산 과세 유예 조치가 납세자 보호라는 관점에서 정당한가에 대해서는 시각에 따라 평가가 다를 수 있습니다.",
    # "네, 여자 코일은 선행발명의 코일과 동일합니다."
]

In [ ]:
# Adaptive 검색과 표준 검색 비교 평가
# 각 쿼리를 두 가지 방법으로 처리하고 결과를 비교합니다.
# 파일 경로
file_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

evaluation_results = evaluate_adaptive_vs_standard(
    pdf_path=file_path,                  # 지식 추출을 위한 소스 문서
    test_queries=test_queries,          # 평가할 테스트 쿼리 목록
    reference_answers=reference_answers  # 비교를 위한 참조 답변
)

# 성능 비교
print(evaluation_results["comparison"])